<a href="https://colab.research.google.com/github/la2015-hw/Group_10/blob/main/KNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"vaishnavichintha","key":"7f7453401b6964fd1e8a85d1dc7c0977"}'}

In [2]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


In [3]:
!kaggle datasets list

ref                                                             title                                                     size  lastUpdated                 downloadCount  voteCount  usabilityRating  
--------------------------------------------------------------  --------------------------------------------------  ----------  --------------------------  -------------  ---------  ---------------  
anandshaw2001/amazon-product-sales-2025                         Amazon Diwali Product Sales 2025                        397394  2025-11-02 06:45:32.877000            950         24  1.0              
ahmeduzaki/global-earthquake-tsunami-risk-assessment-dataset    Global Earthquake-Tsunami Risk Assessment Dataset        16151  2025-10-01 16:35:53.273000          18639        631  1.0              
ahmadrazakashif/bmw-worldwide-sales-records-20102024            BMW Worldwide Sales Records (2010–2024)                 853348  2025-09-20 14:39:45.280000          21308        411  1.0              


In [4]:
!kaggle datasets download -d vaishnavichintha/sdss-tabular-dataset

Dataset URL: https://www.kaggle.com/datasets/vaishnavichintha/sdss-tabular-dataset
License(s): unknown
  0% 0.00/10.7M [00:00<?, ?B/s]
100% 10.7M/10.7M [00:00<00:00, 919MB/s]


In [5]:
!unzip sdss-tabular-dataset.zip

Archive:  sdss-tabular-dataset.zip
  inflating: sdss_gz2_matched_cleaned_balanced.csv  


In [6]:
import pandas as pd
df = pd.read_csv("sdss_gz2_matched_cleaned_balanced.csv")
print(df.head())


             specobjid      dr8objid            dr7objid          ra  \
0   614788390251948032  1.237654e+18  587728665578700971  122.952644   
1  2433194008562919424  1.237665e+18  587739651567845579  231.428600   
2  2080728518915811328  1.237663e+18  587736975809773641  232.551060   
3  1563980851195176960  1.237662e+18  587736619321917717  234.050800   
4  3139079110530197504  1.237666e+18  587739843772940458  217.789000   

         dec     rastring    decstring    sample gz2class  \
0  42.216805  08:11:48.63  +42:13:00.5  original    SBc?m   
1  23.370867  15:25:42.86  +23:22:15.1  original      Ser   
2  27.134108  15:30:12.25  +27:08:02.8  original    SBc2m   
3  30.967339  15:36:12.19  +30:58:02.4  original    SBd2m   
4  20.449600  14:31:09.36  +20:26:58.6  original    SBa2l   

   total_classifications  ...  \
0                     47  ...   
1                     40  ...   
2                     44  ...   
3                     55  ...   
4                     34  ...   

 

In [7]:
df.info()
print(df.columns)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47600 entries, 0 to 47599
Columns: 234 entries, specobjid to broad_class
dtypes: float64(151), int64(78), object(5)
memory usage: 85.0+ MB
Index(['specobjid', 'dr8objid', 'dr7objid', 'ra', 'dec', 'rastring',
       'decstring', 'sample', 'gz2class', 'total_classifications',
       ...
       't11_arms_number_a36_more_than_4_weighted_fraction',
       't11_arms_number_a36_more_than_4_debiased',
       't11_arms_number_a36_more_than_4_flag',
       't11_arms_number_a37_cant_tell_count',
       't11_arms_number_a37_cant_tell_weight',
       't11_arms_number_a37_cant_tell_fraction',
       't11_arms_number_a37_cant_tell_weighted_fraction',
       't11_arms_number_a37_cant_tell_debiased',
       't11_arms_number_a37_cant_tell_flag', 'broad_class'],
      dtype='object', length=234)


In [8]:
# Separate target first
y = df["broad_class"]

# Drop irrelevant columns (IDs, coordinates, etc.)
drop_cols = [
    "specobjid", "dr8objid", "dr7objid",
    "ra", "dec", "rastring", "decstring",
    "sample", "gz2class", "broad_class"  # drop target from features
]

x = df.drop(columns=drop_cols, errors="ignore")

print("Feature shape:", x.shape)
print("Target shape:", y.shape)


Feature shape: (47600, 224)
Target shape: (47600,)


In [9]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
x_scaled = scaler.fit_transform(x)

print("Scaled feature matrix shape:", x_scaled.shape)


Scaled feature matrix shape: (47600, 224)


In [12]:
from sklearn.model_selection import train_test_split

# 70% train, 30% test
x_train, x_test, y_train, y_test = train_test_split(
    x_scaled, y, test_size=0.3, random_state = 38, stratify=y
)

print("Train shape:", x_train.shape, y_train.shape)
print("Test shape:", x_test.shape, y_test.shape)


Train shape: (33320, 224) (33320,)
Test shape: (14280, 224) (14280,)


In [33]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Initialize KNN model
knn = KNeighborsClassifier(n_neighbors=19, weights='uniform')

# Train (fit) the model
knn.fit(x_train, y_train)

# Predict on train and test data
y_train_pred = knn.predict(x_train)
y_test_pred = knn.predict(x_test)

# Evaluate accuracies
train_acc = accuracy_score(y_train, y_train_pred)
test_acc = accuracy_score(y_test, y_test_pred)

print(f"Train Accuracy: {train_acc:.4f}")
print(f"Test Accuracy: {test_acc:.4f}")

# More detailed metrics
print("\nClassification Report:")
print(classification_report(y_test, y_test_pred))

# Confusion Matrix
print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_test_pred))


Train Accuracy: 0.8576
Test Accuracy: 0.8443

Classification Report:
                 precision    recall  f1-score   support

     Elliptical       0.80      0.87      0.83      3570
      Irregular       0.89      0.97      0.93      3570
  Spiral-barred       0.86      0.85      0.86      3570
Spiral-unbarred       0.82      0.69      0.75      3570

       accuracy                           0.84     14280
      macro avg       0.84      0.84      0.84     14280
   weighted avg       0.84      0.84      0.84     14280


Confusion Matrix:
[[3091  152   69  258]
 [  51 3454   22   43]
 [ 162  124 3044  240]
 [ 554  162  386 2468]]
